In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
import glob
import os
import pickle
import json

import cv2
import numpy as np

In [53]:
from skimage.feature import hog

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, make_scorer

In [4]:
DATA_DIR = os.path.join('../data')
VEHICLES = os.path.join(DATA_DIR, 'vehicles')
NON_VEHICLES = os.path.join(DATA_DIR, 'non-vehicles')

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

In [34]:
from sklearn.model_selection import GridSearchCV

In [46]:
class HogFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, color_scheme, orientation, pixels_per_cell, cells_per_block):
        self.color_scheme = color_scheme
        self.orientation = orientation
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
    
    def _hog_feature(self, image):
        return hog(image, orientations=self.orientation, 
                   pixels_per_cell=(self.pixels_per_cell, self.pixels_per_cell), 
                   cells_per_block=(self.cells_per_block, self.cells_per_block), 
                   feature_vector=True)
    
    def fit(self, X, y, **fit_params):
        return self
    
    def transform(self, X):
        X_new = []
        for x in X:
            img = cv2.cvtColor(x, self.color_scheme).astype(np.float32)/255
            rhf = self._hog_feature(img[:, :, 0])
            ghf = self._hog_feature(img[:, :, 1])
            bhf = self._hog_feature(img[:, :, 2])
            X_new.append(np.hstack((rhf, ghf, bhf)))
        return np.array(X_new)
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X)

In [8]:
def read(data_dir):
    for img_path in glob.iglob(os.path.join(data_dir, '*', '*.png')):
        yield cv2.imread(img_path)

In [9]:
vehicles = [feature for feature in read(VEHICLES)]

In [10]:
non_vehicles = [feature for feature in read(NON_VEHICLES)]

In [11]:
data = np.vstack((vehicles, non_vehicles))
labels = np.hstack((np.ones(len(vehicles), np.int32), np.zeros(len(non_vehicles), np.int32)))

In [12]:
len(labels) == len(data)

True

In [13]:
data.shape

(17760, 64, 64, 3)

In [14]:
labels.shape

(17760,)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [16]:
x_train.shape

(14208, 64, 64, 3)

In [47]:
clf = svm.LinearSVC()
scaler = StandardScaler()
extractor = HogFeatureExtractor(cv2.COLOR_BGR2YCrCb, 9, 8, 2)
pipeline = Pipeline([('hog', extractor), ('scaler', scaler), ('svc', clf)])
# pipeline.fit(x_train, y_train)

In [48]:
color_schemes = [cv2.COLOR_BGR2RGB, cv2.COLOR_BGR2HLS, cv2.COLOR_BGR2YCrCb]
orientations = [9, 11, 13]
ppc = [8, 16]
cpb = [2, 3]
param_grid = dict(hog__color_scheme=color_schemes,
                  hog__orientation=orientations, 
                  hog__pixels_per_cell=ppc, 
                  hog__cells_per_block=cpb)

In [54]:
grid = GridSearchCV(pipeline, param_grid=param_grid, 
                    scoring=make_scorer(f1_score))

In [55]:
grid.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('hog', HogFeatureExtractor(cells_per_block=2, color_scheme=36, orientation=9,
          pixels_per_cell=8)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'hog__color_scheme': [4, 52, 36], 'hog__pixels_per_cell': [8, 16], 'hog__cells_per_block': [2, 3], 'hog__orientation': [9, 11, 13]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score), verbose=0)

In [56]:
with open('grid_cv.p', 'wb') as _file:
    pickle.dump(grid, _file)

In [57]:
grid.best_params_

{'hog__cells_per_block': 2,
 'hog__color_scheme': 36,
 'hog__orientation': 13,
 'hog__pixels_per_cell': 8}

In [58]:
grid.best_score_

0.98878342324647428

In [59]:
import pandas as pd

In [60]:
results = pd.DataFrame(grid.cv_results_)

In [61]:
results

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_hog__cells_per_block,param_hog__color_scheme,param_hog__orientation,param_hog__pixels_per_cell,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,62.176049,20.471302,0.963889,1.000000,2,4,9,8,"{'hog__color_scheme': 4, 'hog__pixels_per_cell...",28,0.964721,1.000000,0.960526,1.000000,0.966420,1.000000,4.174857,2.012092,0.002477,0.000000
1,26.473876,11.889537,0.954056,0.998134,2,4,9,16,"{'hog__color_scheme': 4, 'hog__pixels_per_cell...",33,0.953761,0.998416,0.953326,0.998522,0.955080,0.997465,1.245189,0.424366,0.000746,0.000475
2,69.445746,20.699661,0.966355,1.000000,2,4,11,8,"{'hog__color_scheme': 4, 'hog__pixels_per_cell...",24,0.964606,1.000000,0.969581,1.000000,0.964879,1.000000,4.608525,0.125499,0.002284,0.000000
3,27.570767,11.497787,0.955341,0.999261,2,4,11,16,"{'hog__color_scheme': 4, 'hog__pixels_per_cell...",32,0.951390,0.999472,0.955054,0.999261,0.959579,0.999050,0.531900,0.146240,0.003349,0.000172
4,73.622381,21.452239,0.966127,1.000000,2,4,13,8,"{'hog__color_scheme': 4, 'hog__pixels_per_cell...",25,0.967687,1.000000,0.964202,1.000000,0.966491,1.000000,0.468471,0.292642,0.001446,0.000000
5,28.284447,11.943167,0.959135,0.999648,2,4,13,16,"{'hog__color_scheme': 4, 'hog__pixels_per_cell...",31,0.958875,0.999366,0.956815,0.999894,0.961716,0.999683,0.223156,0.197195,0.002010,0.000217
6,47.145064,18.036004,0.982921,1.000000,2,52,9,8,"{'hog__color_scheme': 52, 'hog__pixels_per_cel...",12,0.983000,1.000000,0.983953,1.000000,0.981810,1.000000,3.923475,0.100413,0.000876,0.000000
7,21.745818,10.011371,0.971237,0.999965,2,52,9,16,"{'hog__color_scheme': 52, 'hog__pixels_per_cel...",21,0.966519,1.000000,0.973855,0.999894,0.973336,1.000000,0.262681,0.081042,0.003342,0.000050
8,41.032850,18.546799,0.982438,1.000000,2,52,11,8,"{'hog__color_scheme': 52, 'hog__pixels_per_cel...",14,0.979427,1.000000,0.985862,1.000000,0.982026,1.000000,0.181101,0.109530,0.002643,0.000000
9,22.976230,10.519795,0.973128,0.999930,2,52,11,16,"{'hog__color_scheme': 52, 'hog__pixels_per_cel...",20,0.970272,0.999894,0.973822,1.000000,0.975290,0.999894,0.200215,0.107686,0.002107,0.000050


In [64]:
acc = grid.best_estimator_.score(x_test, y_test)

In [65]:
acc

0.98958333333333337

In [66]:
pred = grid.best_estimator_.predict(x_test)

In [67]:
cm = confusion_matrix(y_test, pred)

In [68]:
cm

array([[1841,   17],
       [  20, 1674]])

In [69]:
with open('classifier.p', 'wb') as _file:
    pickle.dump(grid.best_estimator_, _file)

In [70]:
grid.best_estimator_

Pipeline(memory=None,
     steps=[('hog', HogFeatureExtractor(cells_per_block=2, color_scheme=36, orientation=13,
          pixels_per_cell=8)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])